# Phase 9: Microsoft 365 Copilot Integration

> ⚠️ **Coming Soon**: This lab will be revisited once a **Python sample** is available. The current walkthrough uses the official C# sample for reference.

## Overview

This lab demonstrates how to integrate a **Microsoft Foundry agent** with **Microsoft 365 Copilot** and **Microsoft Teams** using the Microsoft 365 Agents Toolkit.

📚 **Official Sample**: [ProxyAgent-CSharp on GitHub](https://github.com/OfficeDev/microsoft-365-agents-toolkit-samples/tree/dev/ProxyAgent-CSharp)

[![Video Tutorial](https://img.youtube.com/vi/nRuY_YI-Efk/0.jpg)](https://www.youtube.com/watch?v=nRuY_YI-Efk)

## What This Sample Illustrates

| Feature | Description |
|---------|-------------|
| **Foundry to M365 Connection** | Connect a Microsoft Foundry Agent to M365 Copilot |
| **Managed Identity** | Use Agent SDK with managed identity (no secrets to maintain) |
| **SSO Integration** | Set up and use SSO in M365 Copilot & Teams, passing user token to Foundry |
| **Federated Credentials** | Configure SSO with Federated Credentials (no secrets in SSO flow) |

## Architecture Flow

```mermaid
sequenceDiagram
    box "User"
        participant U as Copilot User
    end

    box "Microsoft 365"
        participant M as Microsoft 365 Copilot
    end

    box "Custom Engine Agent"
        participant B as Azure Bot Service
        participant P as Proxy Agent (Agents SDK)
    end

    box "Microsoft Foundry"
        participant A as AI Agent Backend
    end

    U->>M: User prompt (e.g., "Create a report")
    M->>B: Activity (Message)
    B->>P: POST /api/messages (Message)
    P->>B: 202 Accepted
    P-->>M: Start Streaming Session with information
    P->>A: POST /process { prompt }
    A-->>P: { content }
    P-->>M: Stream(content)
    M-->>U: Display result
```

This **proxy pattern** allows you to:
- ✅ Connect existing AI agents to Microsoft 365 Copilot
- ✅ Maintain your AI logic in Microsoft Foundry
- ✅ Provide seamless user experience in Teams and Copilot with SSO
- ✅ Handle authentication and message routing automatically

## Prerequisites

### Required Tools

| Tool | Purpose | Installation |
|------|---------|---------------|
| **.NET SDK 9.0** | Bot runtime | [Download](https://dotnet.microsoft.com/download/dotnet/9.0) |
| **Azure CLI** | Azure authentication | [Install Guide](https://learn.microsoft.com/cli/azure/install-azure-cli) |
| **Microsoft 365 Agents Toolkit CLI** | Deployment orchestration | [Install Guide](https://learn.microsoft.com/en-us/microsoftteams/platform/toolkit/microsoft-365-agents-toolkit-cli#get-started) |
| **Visual Studio Code** | IDE with C# Dev Kit | [Download](https://code.visualstudio.com/) |

### Required Services

- **Microsoft Foundry Project** with a configured agent
- **Microsoft 365 tenant** with Teams or Copilot access
- **Azure subscription** with appropriate permissions

## Solution Architecture

The solution consists of two main components:

### 1. Bot Application (`AzureAgentToM365ATK/`)

.NET 9 bot application that serves as a proxy between Microsoft 365 and Microsoft Foundry.

**Key Features:**
- Connects to Microsoft Foundry Agent Service
- Handles user authentication and SSO
- Manages conversation threads and message routing
- Built on Microsoft 365 Agents SDK

### 2. M365 Agents Toolkit Project (`M365Agent/`)

Infrastructure and configuration for Microsoft 365 integration.

**Includes:**
- Bicep templates for Azure infrastructure deployment
- Teams app manifest configuration
- Environment configuration files
- Automated provisioning and deployment workflows

## Project Structure

```
ProxyAgent-CSharp/
├── AzureAgentToM365ATK/          # C# Bot Application (.NET 9)
│   ├── Program.cs                 # Bot setup and configuration
│   ├── Agents/
│   │   └── AzureAgent.cs          # Microsoft Foundry integration
│   ├── appsettings.json           # Configuration
│   └── appsettings.Development.json  # Local dev settings
│
├── M365Agent/                     # Microsoft 365 Agents Toolkit Project
│   ├── appPackage/                # Teams app package
│   │   ├── manifest.json          # App manifest template
│   │   └── build/                 # Generated manifests
│   ├── infra/                     # Infrastructure as Code
│   │   ├── azure.bicep            # Production deployment
│   │   ├── azure-local.bicep      # Local development
│   │   └── modules/               # Reusable Bicep modules
│   ├── env/                       # Environment variables
│   │   ├── .env.dev               # Azure environment
│   │   └── .env.local             # Local environment
│   ├── m365agents.yml             # Production orchestration
│   └── m365agents.local.yml       # Local orchestration
│
└── README.md                      # Documentation
```

## Quick Start Options

### Option 1: Local Development (Debugging)

Perfect for development and testing with breakpoints and hot reload.

```powershell
# Press F5 in VS Code
# Agent is automatically sideloaded in Teams/M365 Copilot for testing
```

### Option 2: Azure Production Deployment

Deploy your agent to Azure for production or dev environments.

**Using Microsoft 365 Agents Toolkit in VS Code:**
1. Open the **Microsoft 365 Agents Toolkit** extension panel
2. Select **Lifecycle** section
3. Click **Provision** to create Azure resources
4. Click **Deploy** to publish your bot application

**Or using CLI:**
```powershell
cd M365Agent
atk provision --env dev
atk deploy --env dev
```

## What Gets Deployed

When you run `atk provision --env dev`, the following Azure resources are created:

| Resource | Purpose | Naming Pattern |
|----------|---------|----------------|
| **User Assigned Managed Identity** | Bot identity (no passwords!) | `{resourceBaseName}-identity` |
| **App Service Plan** | Compute resources (Linux) | `{resourceBaseName}-plan` |
| **Web App** | Hosts .NET 9 bot application | `{resourceBaseName}-app` |
| **Azure Bot Service** | Azure Bot Service registration | `{resourceBaseName}` |
| **Entra ID App Registration** | SSO authentication | `{botDisplayName}` |
| **OAuth Connection** | SSO token exchange | `SsoConnection` |

## Key Features

### ✅ Single Sign-On (SSO)
- Seamless authentication with federated credentials
- No additional login prompts for users
- Secure token exchange

### ✅ Managed Identity (Production)
- No passwords or secrets to manage
- Automatic credential rotation
- Enhanced security posture

### ✅ Infrastructure as Code
- Repeatable deployments with Bicep
- Version-controlled infrastructure
- Easy environment replication

### ✅ Full Debugging Support
- Set breakpoints in VS Code
- Hot reload for rapid iteration
- Automatic sideloading in Teams/M365 Copilot
- Real-time testing in production environment

## SSO Flow for Agents in Teams & M365 Copilot

```mermaid
sequenceDiagram
  participant User as Agent User
  participant Teams as M365 Copilot
  participant Bot as Azure Bot Service
  participant BF as Azure Bot Service<br/>Token Service
  participant Store as Azure Bot Service<br/>Token Store
  participant AAD as Microsoft Entra ID
  
  User ->> Teams: 1. Send message to Agent
  Teams ->> Bot: Forward message
  Bot ->> BF: 2. Request sign-in link
  BF ->> Bot: Return sign-in link
  Bot ->> Teams: 3. Send OAuth card
  Teams ->> Teams: Check if SSO enabled
  
  alt SSO enabled
    Teams ->> Bot: 4. Send token exchange request
    Bot ->> BF: Forward token exchange
    BF ->> AAD: Exchange token
    alt First time user
      AAD ->> Teams: 5. Request consent
      Teams ->> User: Display consent dialog
      User ->> Teams: Grant consent
      Teams ->> AAD: Consent granted
      AAD ->> BF: Return access token
    else Returning user
      AAD ->> BF: Return access token
    end
    BF ->> Store: 6. Store token
    BF ->> Bot: Token available
    Bot ->> Teams: Process request (authenticated)
  else SSO disabled or consent fails
    Teams ->> User: Display sign-in button
    User ->> Teams: Click sign-in
    Teams ->> AAD: Redirect to sign-in page
    User ->> AAD: Sign in & grant access
    AAD ->> BF: Return access token
    BF ->> Store: Store token
    BF ->> Bot: Token available
    Bot ->> Teams: Process request (authenticated)
  end
  Teams ->> User: Display Agent response
```

**Key Points:**
- **Token Caching**: Azure Bot Service stores tokens for returning users
- **OAuth Card**: Agent receives an OAuth card to deliver the authentication request
- **SSO Experience**: First-time users see a consent dialog, returning users sign in silently
- **Fallback**: If SSO fails, users see traditional sign-in flow
- **Token Exchange**: Uses federated credentials for secure token exchange (no client secrets)

## Microsoft Foundry Configuration

### 1. Create an Agent in Microsoft Foundry Portal

1. Configure the model (GPT-4, GPT-4 Turbo, etc.)
2. Set instructions and personality
3. Add tools and capabilities (Code Interpreter, Functions, etc.)
4. Note the **Agent ID** (starts with `asst_...`)

### 2. Get Connection Details

- **Project Endpoint URL**: `https://your-project.cognitiveservices.azure.com/`
- **Agent ID**: `asst_...`

### 3. Update Configuration

Edit `AzureAgentToM365ATK/appsettings.json`:

```json
{
  "AzureAIFoundryProjectEndpoint": "https://your-project.cognitiveservices.azure.com/",
  "AgentID": "asst_..."
}
```

## Usage Scenarios

### In Microsoft Teams

1. Install the app in Teams (via app package upload)
2. Start a chat with the bot
3. Ask questions or give commands
4. The bot routes requests to your Microsoft Foundry agent
5. Get AI-powered responses with context awareness

### In Microsoft 365 Copilot

1. Access via https://m365copilot.com/
2. Find your agent in the left sidebar
3. Click "Open with Copilot"
4. Use natural language to interact with your Microsoft Foundry agent
5. Seamless integration with other M365 services

## Cost Estimates

### Local Development

| Resource | Cost |
|----------|------|
| Azure Bot Service (F0) | Free (up to 10,000 messages/month) |
| No App Service costs | Running locally |
| **Total** | **~$0/month** |

### Azure Production (Basic)

| Resource | Cost |
|----------|------|
| App Service Plan (B1) | ~$13/month |
| Bot Service (F0) | Free |
| Managed Identity | Free |
| **Total** | **~$13/month** |

### Azure Production (Standard)

| Resource | Cost |
|----------|------|
| App Service Plan (S1) | ~$70/month |
| Bot Service (S1) | ~$0.50 per 1,000 messages |
| Application Insights | ~$2-10/month |
| **Total** | **~$70-100/month** |

## Troubleshooting

### Bot Not Responding

- ✅ Check dev tunnel is running (local) or App Service is started (Azure)
- ✅ Verify bot endpoint in Azure Bot Service configuration
- ✅ Check application logs for errors
- ✅ Verify Microsoft Foundry agent is accessible

### SSO Not Working

- ✅ Check `webApplicationInfo` in app manifest
- ✅ Verify federated credentials in Entra ID app registration
- ✅ Check pre-authorized clients include Teams client IDs
- ✅ Review OAuth connection configuration

### Deployment Failures

- ✅ Verify Azure CLI login and subscription access
- ✅ Check required permissions (Contributor + Application Administrator)
- ✅ Review Bicep deployment errors in Azure Portal
- ✅ Ensure resource names are unique

## Additional Resources

### Official Documentation

- 🔗 [ProxyAgent-CSharp Sample](https://github.com/OfficeDev/microsoft-365-agents-toolkit-samples/tree/dev/ProxyAgent-CSharp)
- 📚 [Microsoft 365 Agents Toolkit Documentation](https://aka.ms/teams-toolkit-docs)
- 📚 [Microsoft 365 Agents SDK](https://github.com/microsoft/agents)
- 📚 [Azure Bot Service Documentation](https://learn.microsoft.com/azure/bot-service/)

### Tutorials & Labs

- 🎓 [Build your own agent with the M365 Agents SDK and Semantic Kernel](https://microsoft.github.io/copilot-camp/pages/custom-engine/agents-sdk/)
- 📹 [Video Tutorial: Microsoft Foundry Agent in M365 Copilot](https://www.youtube.com/watch?v=U9Yv2vjKYbI)

### Microsoft Foundry

- 📰 [Announcing Developer Essentials for Agents and Apps in Microsoft Foundry](https://devblogs.microsoft.com/foundry/announcing-developer-essentials-for-agents-and-apps-in-azure-ai-foundry/)
- 📰 [Microsoft Foundry Agent Service (General Availability)](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/announcing-general-availability-of-azure-ai-foundry-agent-service/4414352)

## Next Steps

Once the **Python sample** is available, this notebook will include:

- ✅ Live code cells for deployment automation
- ✅ Interactive agent configuration
- ✅ Step-by-step provisioning walkthrough
- ✅ Testing and validation cells

---

📚 **Official Sample**: [ProxyAgent-CSharp on GitHub](https://github.com/OfficeDev/microsoft-365-agents-toolkit-samples/tree/dev/ProxyAgent-CSharp)